In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import json
import numpy as np

In [2]:
load_dotenv()

token = os.getenv('TOKEN')

df = pd.read_csv("./dataset.csv", delimiter=';')

def get_paginated_data(url, headers):
    data = []
    while url:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            response_data = response.json()
            data.extend(parse_data(response_data))
            
            link_header = response.headers.get('Link')
            if link_header and 'rel="next"' in link_header:
                next_url = [url_part.split(";")[0].strip("<>") 
                            for url_part in link_header.split(",") 
                            if 'rel="next"' in url_part]
                url = next_url[0] if next_url else None
            else:
                url = None
        else:
            print(f"Erro: {response.status_code}, {response.text}")
            break
    return data

def get_data(url, headers):
    return requests.get(url, headers=headers)

def parse_data(data):
    if isinstance(data, list):
        return data

    if not data:
        return []

    data.pop('incomplete_results', None)
    data.pop('repository_selection', None)
    data.pop('total_count', None)

    namespace_key = list(data.keys())[0]
    return data[namespace_key]

def save_data_to_json(data, filename="data.json"):
    with open(filename, "w") as json_file:
        json.dump(data, json_file, indent=4)

In [46]:
urlApi = "https://api.github.com/repos"

urlRepo = df['URL'][0]

parts = urlRepo.split('https://github.com/')[1].split('/')

owner = parts[0]
repo = parts[1]

url = f"https://api.github.com/repos/{owner}/{repo}/commits"
headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {token}",
    "X-GitHub-Api-Version": "2022-11-28"
}

data = get_paginated_data(url, headers)

In [8]:
total_closed_issues_list = []

url = "https://api.github.com/graphql"
headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {token}",
    "X-GitHub-Api-Version": "2022-11-28"
}

for index, row in df.iterrows():
    urlRepo = row['URL']

    parts = urlRepo.split('https://github.com/')[1].split('/')
    owner = parts[0]
    repo = parts[1]

    query = f"""
    {{
        repository(owner: "{owner}", name: "{repo}") {{
            issues(states: CLOSED) {{
                totalCount
            }}
        }}
    }}
    """

    payload = {
        "query": query
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        data = response.json()

        if data['data']['repository'] is not None:
            total_closed_issues = data['data']['repository']['issues']['totalCount']
            total_closed_issues_list.append(total_closed_issues)
        else:
            error_message = data['errors'][0]['message']
            print(f"Erro: {error_message}")
    else:
        print(f"Erro: {response.status_code}, {response.text}")

print(total_closed_issues_list)

Erro: Could not resolve to a Repository with the name 'incognitochain/incognito-chain'.
Erro: Could not resolve to a Repository with the name 'gfunkmonk/cloudpebble'.
Erro: Could not resolve to a Repository with the name 'Financial-Times/native-ingester'.
Erro: Could not resolve to a Repository with the name 'bisonjp/acme_fitness_demo'.
[10, 0, 0, 3736, 2452, 87, 0, 0, 6642, 1388, 1905, 11027, 0, 832, 0, 2503, 2519, 3801, 485, 3243, 2422, 1424, 0, 4836, 2336, 79, 1020, 3305, 3, 1, 82, 1507, 535, 296, 46, 1651, 45, 326, 23, 0, 0, 39, 2, 983, 1673, 0, 2828, 1461, 0, 2663, 2680, 819, 302, 795, 3002, 1243, 85, 1141, 1021, 4, 304, 0, 2, 446, 1542, 1561, 50, 379, 1231, 588, 56, 757, 3, 968, 1193, 0, 20, 452, 4, 3775, 262, 1048, 947, 333, 1875, 1, 1, 3005, 598, 13, 1737, 543, 195, 16, 30, 81, 18, 161, 0, 4, 181, 15, 0, 0, 1406, 5, 32, 0, 0, 172, 0, 307, 184, 254, 4, 556, 102, 0, 85, 0, 11, 224, 2, 36, 0, 2, 4, 93, 115, 133, 50, 52, 80, 45, 418, 1744, 1, 133, 76, 10, 165, 12, 120, 187, 77, 68,

In [9]:
np_total_closed_issues_list = np.array(total_closed_issues_list)

In [23]:
media = np.average(total_closed_issues_list)
mediana = np.median(total_closed_issues_list)

print("média: ", media)
print("mediana: ", mediana)

indices_maiores_que_media = [index for index, valor in enumerate(total_closed_issues_list) if valor > media]

print("Índices dos valores maiores que a média:", indices_maiores_que_media)

mediana_community_size = df['CommunitySize'].median()

média:  326.6363636363636
mediana:  19.0
Índices dos valores maiores que a média: [3, 4, 8, 9, 10, 11, 13, 15, 16, 17, 18, 19, 20, 21, 23, 24, 26, 27, 31, 32, 35, 43, 44, 46, 47, 49, 50, 51, 53, 54, 55, 57, 58, 63, 64, 65, 67, 68, 69, 71, 73, 74, 77, 79, 81, 82, 83, 84, 87, 88, 90, 91, 104, 115, 134, 135, 155, 342, 371]


In [34]:
df_filtrado = df.iloc[indices_maiores_que_media]

mediana_community_size = df_filtrado['CommunitySize'].median()
media_community_size = df_filtrado['CommunitySize'].mean()

In [35]:
df_filtrado2 = df_filtrado[df_filtrado['CommunitySize'] > media_community_size]

In [36]:
df_filtrado2

,URL,Unnamed: 0,Identifier,_id,FileInfo,NumAuthors,MonNauth,NumActiveMon,EarliestCommitDate,EarliestCommitDateConverted,...,gates,monitors,num_discos,shared_dbs,num_dockers,dockers_raw,structure_raw,ms_depend_graph,avg_size_service,commit_hash
3,https://github.com/mirumee/saleor,17,mirumee/saleor,ObjectId(62901b368b2efbc6e32df464),"{""Ruby"":17,""Perl"":291,""Rust"":1307,""Go"":1,""Sql""...",988.0,"{""2016-01"":12,""2020-07"":48,""2018-12"":46,""2016-...",103,1360687599,12.2.2013,...,[],['jaeger'],2,True,2,"{files:[{'path': '/requirements.txt', 'dbs': [...","{'path': '/.devcontainer/docker-compose.yml', ...","{'nodes': 4, 'edges': 0, 'avg_deps_per_service...",23336.0,99e4b88c3e5545a80f784217e35d6b37cba13611
4,https://github.com/cfpb/hmda-platform,18,cfpb/hmda-platform,ObjectId(629087d38b2efbc6e3a1e16a),"{""Ruby"":5,""Perl"":1,""Scala"":4595,""TypeScript"":3...",300.0,"{""2016-01"":34,""2020-07"":52,""2018-12"":31,""2016-...",104,1361235364,19.2.2013,...,[],[],1,True,7,{files:[{'path': '/kubernetes/keycloak/auth/po...,"{'path': '/docker-compose.yml', 'num_services'...","{'nodes': 3, 'edges': 0, 'avg_deps_per_service...",10849.0,fd7836428253a19d5ca8b7d24a1757305f12efbb
9,https://github.com/golastmile/rasa_nlu,26,golastmile/rasa_nlu,ObjectId(62904e328b2efbc6e3e9d626),"{""Rust"":616,""Python"":2331,""Perl"":3,""other"":551...",867.0,"{""2021-07"":2,""2017-08"":22,""2020-07"":77,""2018-1...",62,1476448069,14.10.2016,...,[],[],0,False,2,"{files:[{'path': '/docs/package.json', 'dbs': ...","{'path': '/docker/docker-compose.yml', 'num_se...","{'nodes': 3, 'edges': 0, 'avg_deps_per_service...",15759.0,221c8d36c77d6fccc7bcdfbbf61727f389501bde
11,https://github.com/samvera/hyrax,29,samvera/hyrax,ObjectId(6290354a8b2efbc6e3903ae5),"{""Ruby"":12962,""Perl"":188,""Python"":9,""PHP"":9,""o...",288.0,"{""2010-09"":5,""2016-01"":22,""2020-07"":13,""2018-1...",141,1269195778,21.3.2010,...,[],[],0,True,1,"{files:[{'path': '/package.json', 'dbs': [], '...","{'path': '/docker-compose.yml', 'num_services'...","{'nodes': 6, 'edges': 7, 'avg_deps_per_service...",16399.0,ed2b4fbb48dd61da78e93c741b86e2f04422b1cd
20,https://github.com/mesosphere/marathon,51,mesosphere/marathon,ObjectId(629090418b2efbc6e3ea1440),"{""Ruby"":1913,""Scala"":2488,""Clojure"":20,""Python...",488.0,"{""2016-01"":33,""2020-07"":4,""2018-12"":23,""2016-1...",98,1372172828,25.6.2013,...,[],[],1,True,10,{files:[{'path': '/tests/shakedown/requirement...,{'path': '/tests/performance/config/docker-com...,"{'nodes': 3, 'edges': 2, 'avg_deps_per_service...",5157.3,90619da70d79a2ecf19368a21c6a695ea981bd74
24,https://github.com/poanetwork/blockscout,59,poanetwork/blockscout,ObjectId(629025398b2efbc6e3bfaf68),"{""other"":11128,""Sql"":19,""Java"":2,""TypeScript"":...",269.0,"{""2021-07"":18,""2020-07"":14,""2018-12"":35,""2021-...",47,1516139779,16.1.2018,...,[],[],0,False,1,{files:[{'path': '/apps/block_scout_web/assets...,"{'path': '/docker-compose/docker-compose.yml',...","{'nodes': 6, 'edges': 3, 'avg_deps_per_service...",27620.0,24f2ff7fd083ec914865aa4ddbbc0990053e6f57
31,https://github.com/dockersamples/example-votin...,85,dockersamples/example-voting-app,ObjectId(629043718b2efbc6e3538734),"{""Ruby"":4,""Lua"":1,""Python"":48,""Go"":7,""PHP"":2,""...",882.0,"{""2016-01"":5,""2020-07"":26,""2018-12"":14,""2016-1...",72,1447617569,15.11.2015,...,[],[],0,True,5,"{files:[{'path': '/vote/requirements.txt', 'db...","{'path': '/docker-compose.yml', 'num_services'...","{'nodes': 4, 'edges': 1, 'avg_deps_per_service...",99.6,3accda954e7c79ca4d90c83100df0d827df0770d
32,https://github.com/openstates/openstates-scrapers,87,openstates/openstates-scrapers,ObjectId(629066908b2efbc6e33e6145),"{""Ruby"":18,""Clojure"":2,""Rust"":60,""Python"":4261...",272.0,"{""2010-09"":4,""2016-01"":9,""2020-07"":5,""2018-12""...",147,1235688657,26.2.2009,...,[],[],0,False,1,"{files:[], structure:{'path': '/docker-compose...","{'path': '/docker-compose.yml', 'num_services'...","{'nodes': 4, 'edges': 0, 'avg_deps_pe

In [38]:
df_filtrado2.to_csv('dataset_filtrado.csv', index=False,  sep=';')

In [51]:
save_data_to_json(data, "github_commits.json")

In [8]:
urlApi = "https://api.github.com/repos"

urlRepo = df_filtrado['URL'][0]

parts = urlRepo.split('https://github.com/')[1].split('/')

owner = parts[0]
repo = parts[1]

url = f"https://api.github.com/repos/{owner}/{repo}/issues"
headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {token}",
    "X-GitHub-Api-Version": "2022-11-28"
}

data = get_paginated_data(url, headers)

In [10]:
save_data_to_json(data, "github_issues.json")